In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[22]:


import os
import sys
import json
import logging
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import dateutil.parser
import operator
import re
import time
from tqdm import tqdm
from dsai import *
import itertools
import webbrowser
import matplotlib.pyplot as plt
import math

#global results

class simulation():
    
    def __init__(self):
        
        os.chdir('C:/Users/DRO2/Desktop/DRO/ACTIVE')
        
        self.jsons_folder_path = './Jsons/'
        
        self.allocation = 'baseline alloc.xlsx'
        self.sw = '0'
        self.ew = '12'
        self.sh = '0'
        self.eh = '23'
        self.day = '9'
        self.dispatch_rules_fn = 'dispatch_rules.xlsx'
        self.avg_stay_f = 'BASELINE.xlsx'
        self.params_fn = 'BASELINE.xlsx'


    def run(self,run_number):
        
        print('Starting Run')
#         telegram.hafeez('Starting Run')
        
        def read_fn(fn):
            try:
                return pd.read_excel(fn)
            except NameError:
                logging.error("File Not Found: %s\n", fn)
                sys.exit(1)
            except Exception:
                logging.error("Error in reading: %s\n", fn)
                sys.exit(1)
                
        def check_incidents(hour, day_id, start_hr, end_hr, day):
            if start_hr > end_hr:
                if end_hr < hour < start_hr:
                    return False
            elif start_hr == end_hr:
                if hour != start_hr:
                    return False
            else:
                if hour < start_hr or hour > end_hr:
                    return False
            # all days
            if day == 9:
                return True

            # weekends
            if day == 8:
                return day_id > 4

            # weekdays
            if day == 7:
                return day_id <= 4

            if start_hr > end_hr:
                day1 = day
                day2 = (day % 7) + 1
                if hour >= start_hr and day_id == day1:
                    return True
                if hour <= end_hr and day_id == day2:
                    return True
            else:
                return day_id == day

        def read_incidents(json_path, start_year, end_year, start_week, end_week, start_hr, end_hr, day):
            # List of json objects, one for each file
            json_list = []
            fn_list = []
            start_json_path = json_path + str(start_year) + "/travelTimeMatrix_" + str(start_year)
            # Need to look at multiple folders as folders are arranged based on years
            if start_year != end_year:
                end_json_path = json_path + str(start_year) + "/travelTimeMatrix_" + str(start_year)
                for w in range(start_week, 53):
                    fn_list.append(start_json_path+"_"+str(w)+".json")
                for w in range(1, end_week+1):
                    fn_list.append(end_json_path + "_" + str(w) + ".json")
            else:
                for w in range(start_week, end_week+1):
                    fn_list.append(start_json_path + "_" + str(w) + ".json")
            # print(fn_list)
            for fn in fn_list:
                try:
                    #print("reading json: " + str(fn))
                    jfn = open(fn)
                    json_list.append(json.load(jfn))
                except NameError:
                    logging.error("File Not Found: %s\n", fn)
                    sys.exit(1)
                except Exception:
                    logging.error("Error in reading: %s\n", fn)
                    sys.exit(1)
            # Filter out the incidents based on start_hr, end_hr, day
            incidents = {}
            for j_obj in json_list:
                for inc_id in j_obj.keys():
                    inc = j_obj.get(inc_id)
                    # TODO: category id is -1 for few of the incidents
                    if inc['incCatId'] < 0:
                        continue
                    date_obj = dateutil.parser.parse(inc['createTime'])
                    hour = date_obj.hour
                    day_id = date_obj.weekday()
                    if check_incidents(hour, day_id, start_hr, end_hr, day):
                        y = date_obj.year
                        day_year = date_obj.timetuple().tm_yday
                        inc['year'] = int(y)
                        inc['day_of_year'] = int(day_year)
                        inc['hour'] = int(hour)
                        incidents[inc_id] = inc
            return incidents

        def read_rules(df):
            # Appliance type that are ored in the rules
            appl_set = set()
            # Dictionary of incident type id with ored appliances and count
            # ex: {1: {FB:1, {A, PA}: 1, {PL, LF, FMV}:1}, 2: {FB:1, {A, PA}:1}}
            inc_type_appl_set_cnt = {}
            # list of incident type ids which need A/PA plus other appliances
            ambulance_plus = []
        #    for inc_type, op, a, pa, pl, lf, fb, fmv in zip(df['IncTypeId'], df['Operator'], df['A'], df['PA'], df['PL'], #KT_PLM
        #                                                    df['LF'], df['FB'], df['FMV']): #KT_PLM
            for inc_type, op, a, pa, pl, plm, lf, lfm, fb, fmv, fmvm in zip(df['IncTypeId'], df['Operator'], df['A'], df['PA'], df['PL'], df['PLM'],#KT_PLM
                                                            df['LF'], df['LFM'], df['FB'], df['FMV'], df['FMVM']): #KT_PLM

                if op == "and":
                    #for cnt, appl in zip([a, pa, pl, lf, fb, fmv], ['A', 'PA', 'PL', 'LF', 'FB', 'FMV']): #KT_PLM
                    for cnt, appl in zip([a, pa, pl, plm, lf, lfm, fb, fmv, fmvm], ['A', 'PA', 'PL', 'PLM', 'LF', 'LFM', 'FB', 'FMV', 'FMVM']): #KT_PLM
                        if not np.isnan(cnt):
                            a_set = frozenset([appl])
                            appl_set.add(a_set)
                            if inc_type not in inc_type_appl_set_cnt:
                                inc_type_appl_set_cnt[inc_type] = {a_set: cnt}
                            else:
                                inc_type_appl_set_cnt.get(inc_type)[a_set] = cnt
                else:
                    appl_list = []
                    ored_count = 0
                    #for cnt, appl in zip([a, pa, pl, lf, fb, fmv], ['A', 'PA', 'PL', 'LF', 'FB', 'FMV']): #KT_PLM
                    for cnt, appl in zip([a, pa, pl, plm, lf, lfm, fb, fmv, fmvm], ['A', 'PA', 'PL', 'PLM', 'LF', 'LFM', 'FB', 'FMV', 'FMVM']): #KT_PLM
                        if not np.isnan(cnt) and cnt > 0:
                            appl_list.append(appl)
                            ored_count = cnt
                    if len(appl_list) > 0:
                        a_set = frozenset(appl_list)
                        appl_set.add(a_set)
                        if inc_type not in inc_type_appl_set_cnt:
                            inc_type_appl_set_cnt[inc_type] = {a_set: ored_count}
                        elif a_set not in inc_type_appl_set_cnt.get(inc_type):
                            inc_type_appl_set_cnt.get(inc_type)[a_set] = ored_count
                        else:
                            c = inc_type_appl_set_cnt.get(inc_type).get(a_set) + 1
                            inc_type_appl_set_cnt.get(inc_type)[a_set] = c

            for inc_type_id in inc_type_appl_set_cnt.keys():
                if frozenset(['A', 'PA']) in inc_type_appl_set_cnt.get(inc_type_id).keys() and                         len(inc_type_appl_set_cnt.get(inc_type_id)) > 1:
                    if (inc_type_id < 7): #KT debug: ambulance_plus only for ems incident type
                        ambulance_plus.append(inc_type_id)

            #print(appl_set, "\n", inc_type_appl_set_cnt, "\n", ambulance_plus, "\n\n\n")
            return appl_set, inc_type_appl_set_cnt, ambulance_plus

        def read_allocation(df):
            allocation_table={}
            #for a, fb, fmv, lf, pl, baseid in zip(df['A'], df['FB'], df['FMV'], df['LF'], df['PL'], df['baseId']): #KT_PLM
            for a, pa, fb, fmv, fmvm, lf, lfm, pl, plm, baseid in zip(df['A'], df['PA'], df['FB'], df['FMV'], df['FMVM'], df['LF'], df['LFM'], df['PL'], df['PLM'], df['baseId']): #KT_PLM
                allocation_table[str(baseid)]={}
                allocation_table[str(baseid)]['A']=round(a)
                allocation_table[str(baseid)]['PA']=round(pa)
                allocation_table[str(baseid)]['FB']=round(fb)
                allocation_table[str(baseid)]['FMV']=round(fmv)
                allocation_table[str(baseid)]['LF']=round(lf)
                allocation_table[str(baseid)]['PL']=round(pl)
                allocation_table[str(baseid)]['PLM']=round(plm) #KT_PLM
                allocation_table[str(baseid)]['LFM']=round(lfm) #KT_PLM
                allocation_table[str(baseid)]['FMVM']=round(fmvm) #KT_PLM
            #print("allocation table: " + str(allocation_table))
            return allocation_table
        
        def simulation(start_year, end_year, start_week, end_week, start_hr, end_hr, day):
            
            lateness = []
            
            at_df = read_fn('./Stay Time/' + self.avg_stay_f)
            p_df = read_fn('./Params/' + self.params_fn)
            jsons_path = self.jsons_folder_path
            
            ## DISPATCH RULES
#             print("Preparing dispatch rules. ", datetime.now())
            r_df = read_fn('./Dispatch Rules/' + self.dispatch_rules_fn)
            appl_set, inc_type_appl_set_cnt, ambulance_plus = read_rules(r_df)
            
            ## ALLOCATION
            print("Preparing allocation table. ", datetime.now())
            self.allocation_name = re.findall(r'^([^\d]*)',self.allocation)[0].strip()
            a_df = read_fn('./Allocations/' + self.allocation)
            allocation_table = read_allocation(a_df)
            
            res3 = 0
            res4 = 0
            med_res3 = 0
            med_res4 = 0
            
            DEF_AT_SC_TIME = {}
            DEF_AT_HOSP_TIME = {}
            
            for a, at_s, at_h in zip(at_df['appl_type'], at_df['at_scene'], at_df['at_hosp']):
                DEF_AT_SC_TIME[a] = at_s
                DEF_AT_HOSP_TIME[a] = at_h
            
            FIRE_VEHICLES=set(['PL','PLM','LF','LFM','FB','FMV','FMVM'])
            AMBULANCES=set(['A','PA'])

            
            bounded_time = p_df['bounded_time'][0]
            p12_bounded_time = p_df['p12_bounded_time'][0]
            p3_bounded_time = p_df['p3_bounded_time'][0]
            medical_fire_bounded_time = p_df['medical_fire_bounded_time'][0]
            amb_bounded_time = p_df['amb_bounded_time'][0]
            upper_limit = p_df['upper_limit'][0]

            if start_year > end_year:
                logging.error("Start year (%s) is greater than the end year (%s).", start_year, end_year)
                sys.exit(0)

            if day > 6:
                if start_hr > end_hr:
                    logging.error("Start hour (%s) is greater than the end hour (%s).", start_hr, end_hr)
                    sys.exit(0)

            #print("Reading incidents. ", datetime.now())
            # set of valid incidents
            incidents = read_incidents(jsons_path, start_year, end_year, start_week, end_week, start_hr, end_hr, day)
            incident_df = pd.DataFrame(incidents).T
            incident_df.to_excel('./Results/Incidents.xlsx',index = False)
            #print("     Number of incidents: ", len(incidents))


            event_obj=[]
            event_incr=[]
            opt_res_time={}
            inc_served={}
            inc_count={}
            non_served={}
            incident_res_time=[]
            inc_servedNoAmb = {} #KT_NoAmbKPI
            inc_servedNoAmb[str(1)] = 0 #KT_NoAmbKPI
            inc_servedNoAmb[str(2)] = 0 #KT_NoAmbKPI

            for inc_id in incidents:
                #if inc_id>'201712310350':
                    #break
                if int(incidents[inc_id]['incCatId']) not in inc_type_appl_set_cnt:
                    continue
                inc_time=datetime.strptime(incidents[inc_id]['createTime'][0:19], '%Y-%m-%dT%H:%M:%S')
                obj={'time':inc_time, 'id':inc_id, 'flag':0}
                event_obj.append(obj)
            #print("TOTAL COUNT OF INCIDENTS "+str(len(event_obj)))
            event_obj.sort(key=operator.itemgetter('time'))

            
            ## LONG PART IH add
            total_incident_count = len(incident_df)
            current_time = time.time()
            current_number = 0
#             telegram.hafeez(str(total_incident_count) + ' number of cases')
            print(str(total_incident_count) + ' number of cases')
            print(str(int(total_incident_count * 2.21)) + ' on the progress bar')
            
            for obj in tqdm(event_obj):
                if time.time() - current_time > 1200:
                    
                    telegram.mahadhir(self.allocation_name + str(round((current_number+1) / (total_incident_count+0.0001) * 100,2)) + '% completed.')
                    telegram.hafeez(self.allocation_name + str(round((current_number+1) / (total_incident_count+0.0001) * 100,2)) + '% completed.')
                    current_time = time.time()
                    
                if obj['flag']==0:
                    
                    current_number += 1
                    #print("incID: " + str(obj['id']))
                    incident=incidents[obj['id']]
                    inc_type=incident['incCatId']
                    #print("inc_type " + str(inc_type)) #KT learn
                    if str(inc_type) not in opt_res_time:
                        opt_res_time[str(inc_type)]=[]
                    if str(inc_type) not in inc_served:
                        inc_served[str(inc_type)]=0
                    if str(inc_type) not in inc_count:
                        inc_count[str(inc_type)]=1
                    else:
                        inc_count[str(inc_type)]+=1
                    inc_time=obj['time']
                    inc_resp_time=60.0
                    inc_r_time={'fire':60.0,'amb':60.0}
                    desp_rule=dict(inc_type_appl_set_cnt[inc_type])
                    #print("Despatch rule: "+ str(desp_rule)) #KT learn
                    at_scene_dur={}
                    at_hosp_dur={}
                    to_hosp_dur={}
                    appl_data={}
                    for callsign in incident['applList']:
                        appl=re.findall("[a-zA-Z]+", callsign)[0]
                        #if appl=='PA':
                            #appl='A'
                        appl_data[appl]=incident['applList'][callsign]['isEnrouteHosp']
                        at_scene_dur[appl]=incident['applList'][callsign]['atSceneDur']
                        at_hosp_dur[appl]=incident['applList'][callsign]['atHospDur']
                        to_hosp_dur[appl]=incident['applList'][callsign]['toHospDur']

                        if (appl == "PL" or appl == "LF" or appl == "FMV"):
                            appl_data[appl+'M']=incident['applList'][callsign]['isEnrouteHosp']
                            at_scene_dur[appl+'M']=incident['applList'][callsign]['atSceneDur']
                            at_hosp_dur[appl+'M']=incident['applList'][callsign]['atHospDur']
                            to_hosp_dur[appl+'M']=incident['applList'][callsign]['toHospDur']

                    #print("at scene:" + str(at_scene_dur) + "; at hosp" + str(at_hosp_dur) + "; to hosp" + str(to_hosp_dur))

                    #print("at scene dur "+str(at_scene_dur))
                    #print("at hosp dur "+str(at_hosp_dur))
                    #print("to hosp dur "+str(to_hosp_dur))
                    #print("ret from hosp "+str(ret_hosp_dur))
                    veh_count=0
                    for appl_set in desp_rule:
                        veh_count+=int(desp_rule[appl_set])
                    #print("veh_count:" + str(veh_count))  #KT learn
                    incident['dispFacSetList'].sort()
                    fac_list=[]
                    fac_id_list=[]
                    for fac_type in incident['dispFacSetList']:
                        #print("fac_type/facSet: " + str(fac_type)) #KT learn
                        for item in range(len(incident['dispNearestFacilities'][str(fac_type)])):
                            if incident['dispNearestFacilities'][str(fac_type)][item]['id'] not in fac_id_list:
                                fac_id_list.append(incident['dispNearestFacilities'][str(fac_type)][item]['id'])
                                incident['dispNearestFacilities'][str(fac_type)][item]['fac_type']=str(fac_type)
                                fac_list.append(incident['dispNearestFacilities'][str(fac_type)][item])
                    fac_list.sort(key=operator.itemgetter('response_time'))

                    for fac in fac_list:
                        if veh_count<=0:
                            break
                        #print("Searching in base with id: "+str(fac['id'])) #KT learn
                        base_id=fac['id']
                        res_time=fac['response_time']
                        ret_to_base=fac['return_base_time']
                        if incident['isEnrouteHosp'] and 'return_base_from_hosp_time' in fac:
                            ret_to_base_from_hosp=fac['return_base_from_hosp_time']
                        else:
                            ret_to_base_from_hosp=None
                        for appl_set in desp_rule:
                            #print("appl_set:" + str(appl_set)) #KT learn
                            #set_count=int(desp_rule[appl_set])
                            #creating a new set to see if few appls are present in the base and have info in data
                            #if appl present in base and in data, gets more priority
                            #if desp_rule[appl_set]==0:
                                #continue
                            for i_ in range(0,int(desp_rule[appl_set])):
                            #while desp_rule[appl_set]>0:
                                appl_set_new=list()
                                for appl in appl_set:
                                    if (str(base_id) in allocation_table and appl in allocation_table[str(base_id)]) and allocation_table[str(base_id)][appl]>0:
                                        for callsign in incident['applList']:
                                            if re.findall("[a-zA-Z]+", callsign)[0] == appl:
                                                appl_set_new.append(appl)
                                #if len(appl_set_new)==0:
                                    #appl_set_new=appl_set
                                if len(appl_set_new)<len(appl_set):
                                    for appl in appl_set:
                                        if appl not in appl_set_new:
                                            appl_set_new.append(appl)

                                for appl in appl_set_new:
                                    if veh_count<=0:
                                        break
                                   #print(appl)
                                    if str(base_id) in allocation_table and (appl in allocation_table[str(base_id)]) and allocation_table[str(base_id)][appl]>0:
                                        if (str(inc_type)=='1' or str(inc_type)=='2') and appl in FIRE_VEHICLES and res_time>medical_fire_bounded_time:
                                            veh_count-=1
                                            desp_rule[appl_set]-=1
                                            break
                                        #if appl=='PL' and fac['fac_type']!=1:
                                        #if appl=='PL' and fac['fac_type']!='1': #KT debug: should be '1' instead of 1 #KT_PLM
                                        #if appl is PL or PLM, do not need to check non FS, however 45 is exception because it is big enough
                                        if ((appl=='PL' or appl=='PLM') and (fac['fac_type']!='1') and base_id != 45) : #KT debug: should be '1' instead of 1 #KT_PLM
                                            continue
                                       #print("allocation of base id "+str(base_id)+" "+str(allocation_table[str(base_id)]))
                                        allocation_table[str(base_id)][appl]-=1 #reduce count from base
                                        veh_count-=1
                                        #set_count-=1.0
                                        desp_rule[appl_set]-=1
                                        #resp time of incident is the time when first vehicle arrives
                                        if inc_resp_time>res_time:
                                            inc_resp_time=res_time
                                        if appl in FIRE_VEHICLES and inc_r_time['fire']>res_time:
                                            inc_r_time['fire']=res_time
                                        if appl in AMBULANCES and inc_r_time['amb']>res_time:
                                            inc_r_time['amb']=res_time
                                       #print("appl "+appl+" base "+str(base_id)+" time: "+str(res_time)+" veh count "+str(veh_count))
                                       #print("allocation of base id now is "+str(base_id)+" "+str(allocation_table[str(base_id)]))
                                        #get duration at scene, hosp etc time from data to calculate turn-around time
                                        t_a_time=0.0
                                        if appl in appl_data and appl_data[appl]:
                                            #does go to hosptial
                                            """
                                            if str(base_id) in ret_hosp_dur[appl]: 
                                               #print(str(res_time)+"+"+str(at_scene_dur[appl])+"+"+str(to_hosp_dur[appl])+"+"+str(at_hosp_dur[appl])+"+"+str(ret_hosp_dur[appl][str(base_id)]))
                                                t_a_time=inc_time+timedelta(minutes=(res_time+at_scene_dur[appl]+to_hosp_dur[appl]+at_hosp_dur[appl]+ret_to_base_from_hosp))
                                            else:
                                               #print(str(res_time)+"+"+str(at_scene_dur[appl])+"+"+str(to_hosp_dur[appl])+"+"+str(at_hosp_dur[appl])+"+ sum of "+str(res_time)+" "+str(to_hosp_dur[appl]))
                                                t_a_time=inc_time+timedelta(minutes=(res_time+at_scene_dur[appl]+to_hosp_dur[appl]+at_hosp_dur[appl]+res_time+to_hosp_dur[appl]))
                                            """
                                            if ret_to_base_from_hosp == None:
                                                ret_to_base_from_hosp = (res_time + to_hosp_dur[appl])/2
                                            t_a_time=inc_time+timedelta(minutes=(res_time+at_scene_dur[appl]+to_hosp_dur[appl]+at_hosp_dur[appl]+ret_to_base_from_hosp))
                                        elif appl in appl_data and not appl_data[appl]: #does not go to hospital
                                           #print(str(res_time*2)+"+" + str(at_scene_dur[appl]))
                                            t_a_time=inc_time+timedelta(minutes=(res_time + at_scene_dur[appl] + ret_to_base))
                                        ## CHECK HERE FOR PA
                                        elif appl not in appl_data:
                                            #if (appl=='A' or appl=='FMV') and 'PA' in appl_data: #KT_PLM
                                            if (appl=='A' or appl=='FMV' or appl=='FMVM') and 'PA' in appl_data: #KT_PLM
                                                if appl_data['PA']:
                                                    if ret_to_base_from_hosp == None:
                                                        ret_to_base_from_hosp = (res_time + to_hosp_dur['PA'])/2
                                                    t_a_time=inc_time+timedelta(minutes=(res_time+at_scene_dur['PA']+to_hosp_dur['PA']+at_hosp_dur['PA']+ret_to_base_from_hosp))
                                                    """                            
                                                    if str(base_id) in ret_hosp_dur['PA']:
                                                    #print(str(res_time)+"+"+str(at_scene_dur['PA'])+"+"+str(to_hosp_dur['PA'])+"+"+str(at_hosp_dur['PA'])+"+"+str(ret_hosp_dur['PA'][str(base_id)]))
                                                    t_a_time=inc_time+timedelta(minutes=(res_time+at_scene_dur['PA']+to_hosp_dur['PA']+at_hosp_dur['PA']+ret_hosp_dur['PA'][str(base_id)])) 
                                                    else:
                                                    #print(str(res_time)+"+"+str(at_scene_dur['PA'])+"+"+str(to_hosp_dur['PA'])+"+"+str(at_hosp_dur['PA'])+"+"+str(res_time))
                                                    t_a_time=inc_time+timedelta(minutes=(res_time+at_scene_dur['PA']+to_hosp_dur['PA']+at_hosp_dur['PA']+res_time))
                                                    """
                                                else:
                                                   #print(str(res_time*2)+"+" + str(at_scene_dur['PA']))
                                                    t_a_time=inc_time+timedelta(minutes=(res_time + at_scene_dur['PA'] + ret_to_base))
                                            elif len(appl_data)!=0:
                                                temp_a=list(at_scene_dur.keys())[0]
                                                #if appl_data[temp_a] and (appl=='A' or appl=='PA' or appl=='FMV'): #KT_PLM
                                                if appl_data[temp_a] and (appl=='A' or appl=='PA' or appl=='FMV' or appl=='FMVM'): #KT_PLM
                                                    if ret_to_base_from_hosp == None:
                                                        ret_to_base_from_hosp = (res_time + to_hosp_dur[temp_a])/2
                                                   #print(str(res_time)+"+"+str(at_scene_dur[temp_a])+"+"+str(to_hosp_dur[temp_a])+"+"+str(at_hosp_dur[temp_a])+"+"+str(ret_hosp_dur[temp_a][str(base_id)]))
                                                    t_a_time=inc_time+timedelta(minutes=(res_time+at_scene_dur[temp_a]+to_hosp_dur[temp_a]+at_hosp_dur[temp_a]+ret_to_base_from_hosp)) 
                                                else:
                                                   #print(str(res_time*2)+"+" + str(at_scene_dur[temp_a]))
                                                    t_a_time=inc_time+timedelta(minutes=(res_time + at_scene_dur[temp_a] + ret_to_base))
                                            else:
                                               #print(str(res_time*2) + str(DEF_AT_SC_TIME[appl]))
                                                t_a_time=inc_time+timedelta(minutes=(res_time + DEF_AT_SC_TIME[appl] + ret_to_base))
                                        #add obj to event_obj
                                       #print("turn around time "+str(t_a_time))
                                        new_obj={'time':t_a_time, 'id':len(event_incr), 'flag':1}
                                        insert_flag=0
                                        for index, event in enumerate(event_obj):
                                            if t_a_time <= event['time']:
                                                event_obj.insert(index, new_obj)
                                                insert_flag=1
                                                break
                                        if insert_flag==0:
                                            event_obj.append(new_obj)
                                        event_incr.append({'appl':appl, 'baseId':base_id})
                                    #if set_count==0.0:
                                    if desp_rule[appl_set]==0:
                                        break
                    #if veh_count !=0: #not served, append to opt_res_time, orig_res_time
                    #inc_resp_time+=5.0
                    opt_res_time[str(inc_type)].append(inc_resp_time)
                    incident_res_time.append([obj['id'], inc_type, incident['callPstatus'], inc_resp_time, inc_r_time['fire'], inc_r_time['amb']])

                    #if (incident['callPstatus']==1.2 or incident['callPstatus']==2) and inc_resp_time<=p12_bounded_time:
                    #KT_NoAmbKPI: specially save KPI for P1+CR without considering amb arrival time
                    ambulance_plus=[1,2,3]
                    if (int(inc_type) == 4 or int(inc_type) == 5) and inc_resp_time<=p12_bounded_time:
                        inc_served[str(inc_type)]+=1
                    #elif (incident['callPstatus']==3 or int(inc_type)==7) and inc_resp_time<=p3_bounded_time:
                    elif (int(inc_type)==6 or int(inc_type)==7) and inc_resp_time<=p3_bounded_time:
                        inc_served[str(inc_type)]+=1
                    elif int(inc_type) in ambulance_plus:
                        if (inc_r_time['fire']<=bounded_time and inc_r_time['amb']<=medical_fire_bounded_time) or (inc_r_time['fire']>bounded_time and inc_r_time['amb']<=bounded_time):
                            inc_served[str(inc_type)]+=1
                        if (inc_resp_time<=bounded_time):
                            inc_servedNoAmb[str(inc_type)]+=1 #KT_NoAmbKPI
                        else:
                            lateness.append(str(obj['id']))
                            # lateness_class.append(str(inc_type))
                    elif inc_resp_time<=bounded_time:# and int(inc_type) not in ambulance_plus and 
                        inc_served[str(inc_type)]+=1
                    if inc_resp_time==60.0:
                        #non_served+=1
                        #if inc_type not in non_served:
                        if str(inc_type) not in non_served: #KT debug: should be str(inc_type) instead of inc_type
                            non_served[str(inc_type)]=1
                        else:
                            non_served[str(inc_type)]+=1
                            
                    if inc_resp_time < 11:
                        res3 += 1
                        res4 += 1
                        if inc_type in [1,2,3,4,5,6,7]:
                            med_res3 += 1
                            med_res4 += 1
                    else:
                        res4 += 1
                        if inc_type in [1,2,3,4,5,6,7]:
                            med_res4 += 1

                       #print(" NOT SERVED pstatus "+str(incident['pstatus'])+" Inc Type "+str(inc_type)+" resp time "+str(inc_resp_time))

                else: #flag==1, incr count in allocation table
                    base_id=event_incr[obj['id']]['baseId']
                    appl=event_incr[obj['id']]['appl']
                    allocation_table[str(base_id)][appl]+=1
                   #print("allocation of base id now is "+str(base_id)+" "+str(allocation_table[str(base_id)]))


            #print("Getting Results:")
            res1=0
            res2=0

            ## HAFEEZ inc_type
            med_res1 = 0
            med_res2 = 0

            for inc_type in inc_count:
                res1+=inc_served[inc_type]
                res2+=inc_count[inc_type]
            ## HAFEEZ inc_type
            for inc_type in ['1','2','3','4','5','6','7']:
                try:
                    med_res1+=inc_served[inc_type]
                    med_res2+=inc_count[inc_type]
                except: pass

            #print("Percentage of medical incidents served are "+str(med_res1/med_res2*100)+", total count is: "+str(med_res2))
            #print("Percentage of incidents served are "+str(res1/res2*100)+", total count is: "+str(res2))

            #print("Incidents not served because no vehicles availble: "+str(non_served))
            res_kpi = {'Allocation':self.allocation_name, 'Start Wk': self.sw, 'End Wk': self.ew, 'Start Hr': self.sh, 'End Hr': self.eh, 'Day': self.day, 'Dispatch Rules': self.dispatch_rules_fn, 'Type': 'KPI'}
            #res_count = {'Allocation':self.allocation_name, 'Timing': self.timing, 'Dispatch Rules': self.dispatch_rules_fn, 'Type': 'Count'}
            #inc_type_to_cat = {'1': 'p1+', '2': 'p1+', '3':'p1+', '4':'p1', '5':'p2', '6':'p3', '7':'p4', '0':'fire'}
            inc_type_to_cat = {'1': 'P1+ < 8', '2': 'P1+ < 8', '3':'P1+ < 8', '4':'P1 < 11', '5':'P2 < 11', '6':'P3 < 15', '7':'P4', '0':'Fire < 8', '99':'P1+ < 8'} #KT_NoAmbKPI
            cnt_fire=0
            cnt_fire_raw=0
            cnt_fire_total=0
            cnt_p1plus=0
            cnt_p1plusNoAmb=0
            cnt_p1plus_raw=0
            cnt_p1plus_total=0
            cnt_p1plus_totalNoAmb=0
            for inc_type in inc_count:
                #KT_NoAmbKPI
                if int(inc_type)==1 or int(inc_type)==2:
                    cnt_p1plusNoAmb = cnt_p1plusNoAmb + inc_servedNoAmb[inc_type]
                if int(inc_type)==3:
                    cnt_p1plusNoAmb = cnt_p1plusNoAmb + inc_served[inc_type]
                #KT_NoAmbKPI

                if int(inc_type)==1 or int(inc_type)==2 or int(inc_type)==3:
                    cnt_p1plus = cnt_p1plus + inc_served[inc_type]
                    cnt_p1plus_total = cnt_p1plus_total + inc_count[inc_type]
                    cnt_p1plus_totalNoAmb = cnt_p1plus_totalNoAmb + inc_count[inc_type]
                elif int(inc_type) <= 7:
                    cat = inc_type_to_cat[str(inc_type)]
                    #res_count[cat] = inc_count[inc_type]
                    res_kpi[cat] = (round(inc_served[inc_type]/(inc_count[inc_type]+0.0001)*100, 4), inc_count[inc_type])
                else:
                    #cat = inc_type_to_cat['0']
                    cnt_fire = cnt_fire + inc_served[inc_type]
                    cnt_fire_total = cnt_fire_total + inc_count[inc_type]
#             res_count['Fire < 8'] = cnt_fire_total
            res_kpi['Fire < 8'] = (round(cnt_fire/(cnt_fire_total+0.0001)*100, 4), cnt_fire_total)
#             res_count['P1+ < 8'] = cnt_p1plus_totalNoAmb
            res_kpi['P1+ < 8'] = (round(cnt_p1plusNoAmb/(cnt_p1plus_totalNoAmb+0.0001)*100, 4), cnt_p1plus_totalNoAmb)
#             res_count['P1+ < 8 & Amb < 11'] = cnt_p1plus_total
            res_kpi['P1+ < 8 & Amb < 11'] = (round(cnt_p1plus/(cnt_p1plus_total+0.0001)*100, 4), cnt_p1plus_total)
#             res_count['Medical < 11'] = med_res4
            res_kpi['Medical < 11'] = (round(med_res3/(med_res4+0.0001)*100,4), med_res4)
#             res_count['Overall < 11'] = res4
            res_kpi['Overall < 11'] = (round(res3/(res4+0.0001)*100,4), res4)
#             res_count['Served in time'] = res2
            res_kpi['Served in time'] = (round(res1/(res2+0.0001)*100,4), res2)
#             res_kpi['overall_medical']=[]
#             res_kpi['overall_medical'].append(med_res2)
            # res_kpi['overall_medical'].append(round(med_res1/med_res2*100,2))
            return res_kpi,lateness

        #key-> name of the allocation file, array values-> range of days for testing
        #params={'16-26-0-23-0':['16-26-0-23-0', '26-36-0-23-0', '36-52-0-23-0']} #KT
        
        res_kpi,lateness = simulation(2018, 2018, int(self.sw), int(self.ew), int(self.sh), int(self.eh), int(self.day))
        
        self.results[run_number].append(res_kpi)
        self.lateness[run_number].append(lateness)
#         self.results[run_number].append(res_count)

    
    
    def ending(self):
        self.end_time = time.time()
        self.total_time = self.end_time - self.start_time
        self.hours = int(np.floor(self.total_time / 3600))
        self.minutes = int(np.floor((self.total_time - 3600 * self.hours) / 60))
        self.seconds = int(np.floor(self.total_time - 3600 * self.hours - 60 * self.minutes))
        telegram.mahadhir('Completed: ' + str(self.hours) + 'h ' + str(self.minutes) + 'm ' + str(self.seconds) + 's')
        telegram.hafeez('Completed: ' + str(self.hours) + 'h ' + str(self.minutes) + 'm ' + str(self.seconds) + 's')
        webbrowser.open('http://localhost:3000/')
        
    def setup(self):
        
        self.permutations = pd.DataFrame(list(itertools.product(os.listdir('Allocations'), os.listdir('Timings'), os.listdir('Stay Time'), os.listdir('Params'), os.listdir('Dispatch Rules'))))
        self.permutations.columns = ['Allocations','Timings','Stay Time','Params','Dispatch Rules']
        self.permutations[['SW','EW','SH','EH','DAY']] = self.permutations['Timings'].apply(lambda x: x[:-5]).str.split('-',expand=True)
        to_remove = ['Timings']
        
        for col in self.permutations:
            if len(self.permutations[col].unique().tolist()) < 2:
                to_remove.append(col)
                
        self.permutations = self.permutations.drop(to_remove, axis=1)
        self.permutations['Run'] = 0
        self.permutations.to_excel('Run.xlsx')
    
    def full_run(self):
        
        self.start_time = time.time()
        self.results = {x:[] for x in range(100)}
        self.lateness = {x:[] for x in range(100)}
        
        self.edited_permutations = pd.read_excel('Run.xlsx').to_dict('records')
        
        self.number_of_runs = len(self.edited_permutations)
        
        print('Number of runs: ' + str(self.number_of_runs))
        telegram.mahadhir('Number of runs: ' + str(self.number_of_runs))
        telegram.hafeez('Number of runs: ' + str(self.number_of_runs))
        
        for permutation in self.edited_permutations:
            
            telegram.mahadhir('\n'.join([str(i) for i in permutation.values()]))
            telegram.hafeez('\n'.join([str(i) for i in permutation.values()]))
            
            try:
                if permutation['Allocations'] != self.allocation:
                    self.allocation = permutation['Allocations']
                    self.allocation_name = re.findall(r'^([^\d]*)',self.allocation)[0].strip()
            except: pass
                
            try:
                if permutation['SW'] != self.sw:
                    self.sw = permutation['SW']
            except: pass
            
            try:
                if permutation['EW'] != self.ew:
                    self.ew = permutation['EW']
            except: pass
            
            try:
                if permutation['SH'] != self.sh:
                    self.sh = permutation['SH']
            except: pass
            
            try:
                if permutation['EH'] != self.eh:
                    self.eh = permutation['EH']
            except: pass
            
            try:
                if permutation['DAY'] != self.day:
                    self.day = permutation['DAY']
            except: pass        
            
            try:
                if permutation['Dispatch Rules'] != self.dispatch_rules_fn:
                    self.dispatch_rules_fn = permutation['Dispatch Rules']
            except: pass
            
            try:
                if permutation['Stay Time'] != self.avg_stay_f:
                    self.avg_stay_f = permutation['Stay Time']
            except: pass
            
            try:
                if permutation['Params'] != self.params_fn:
                    self.params_fn = permutation['Params']
            except: pass
            
            self.run(int(permutation['Run']))
#             except Exception as e: 
# #                 telegram.mahadhir(e)
# #                 telegram.hafeez(e)
#                 print(e)
#                 webbrowser.open('http://localhost:3000/facilitylists')
            
            #results = self.results
            #%store results
            
#         pd.DataFrame(self.results).to_excel('./Results/Clean.xlsx')
            with open('results.txt','w') as outfile:
                json.dump(self.results,outfile)

            with open('lateness.txt','w') as outfile2:
                json.dump(self.lateness,outfile2)
            
        self.ending()
        


# In[23]:


sim = simulation()
sim.setup()


# In[24]:


sim.full_run()


# In[368]:


with open('results.txt') as results:
    runs = json.load(results)
to_get = ['P1+ < 8', 'P1 < 11', 'P2 < 11' , 'P3 < 15', 'Medical < 11', 'Fire < 8', 'Served in time']
df_kpi = pd.DataFrame(columns = to_get)
df_count = pd.DataFrame(columns = to_get)
def get_kpi(run_no):
    kpi_list = []
    try:
        for x in to_get:
            temp = list(i[x] for i in runs[run_no])
            kpi_list.append(sum(a*b for a,b in temp)/sum(b for a, b in temp))
        df_kpi.loc[int(run_no)] = kpi_list
    except:
        pass
    return df_kpi

def get_count(run_no):
    count_list = []
    try:
        for x in to_get:
            temp = list(i[x] for i in runs[run_no] if i[x] != 0)
            count_list.append(sum(b for a, b in temp))
        df_count.loc[int(run_no)] = count_list
    except:
        pass
    return df_count

for _ in runs.keys():
    get_kpi(_)
    get_count(_)
    
df_count.to_clipboard()


# In[57]:


def json_inc_mapper():
    global json_mapping
    json_mapping = {}
    for file in os.listdir('Jsons/2018'):
        incJson = json.load(open('Jsons/2018/'+file))
        for inc in incJson:
            json_mapping[inc] = incJson[inc]
            
def inc_get_coords(x):
    return [json_mapping[x]['coordinate']['lat'],json_mapping[x]['coordinate']['lng']]

def inc_get_time(x):
    return json_mapping[x]['createTime']

def lateness():
    global lateness_dfs
    json_inc_mapper()
    latenessJson = json.load(open('lateness.txt'))
    lateness_dfs = []
    for runNum in latenessJson:
        df = pd.DataFrame([item for sublist in latenessJson[runNum] for item in sublist],columns=['inc'])
        df['Lat'] = df['inc'].apply(lambda x: inc_get_coords(x)[0])
        df['Long'] = df['inc'].apply(lambda x: inc_get_coords(x)[1])
        df['Time'] = df['inc'].apply(inc_get_time)
        lateness_dfs.append(df)

# os.chdir('DRO/ACTIVE')
lateness()
lateness_dfs[1].to_csv('lateness.csv',index=False)
late = lateness_dfs[1]
        
def lateness_metric(lat,long):
    lats = late['Lat'].tolist()
    longs = late['Long'].tolist()
    metric = 0
    for late_number in range(len(lats)):
        dist = 111.33 * ((lats[late_number]-lat)**2 + (longs[late_number]-long)**2)**0.5
        metric += math.atan(-100*(dist-3.77))/math.pi + 0.5
    return metric

dro.bases['Metric'] = np.vectorize(lateness_metric)(dro.bases['Lat'],dro.bases['Long'])
results = dro.bases.sort_values('Metric',ascending=False).loc[dro.bases['Type'].isin(['FP','FS'])]


# In[51]:


fig, ax = plt.subplots(1,1)
ax.scatter(late['Long'],late['Lat'],s=10,alpha=0.5)
ax.scatter(results['Long'],results['Lat'],s=results['Metric']/10)


# In[46]:


results['##A'] = results['#A'] + results['#PA']
def available(ambs,metric,base_type):
    if base_type == 'FP':
        max_ambs = 2
    elif base_type == 'FS':
        max_ambs = 4
    else: max_ambs = 0
    if max_ambs > ambs: return True
    else: return False
    
results['Available'] = results[['##A','Metric','Type']].apply(lambda x: available(x[0],x[1],x[2]),axis=1)


# In[47]:


n = 4
accepted_bases = {}
accepted_names = []
results_dict = results.loc[results['Available']].set_index('Complete Name').to_dict('index')
for base_name in results_dict:
    base_info = results_dict[base_name]
    if len(accepted_bases) < n:
        cannot = 0
        for accepted_name in accepted_bases:
            accepted = accepted_bases[accepted_name]
            dist = geo.dist(base_info['Lat'],base_info['Long'],accepted['Lat'],accepted['Long'])
            if dist < 5: cannot += 1
        if cannot == 0:
            accepted_bases[base_name] = base_info
            accepted_names.append(base_name)
if len(accepted_bases) < n:
    print('Not Possible')


# In[52]:


accepted_names


# In[2]:


import sys
sys.path


# In[3]:


import dsai


# In[ ]:



